In [1]:
import psycopg2
from tkinter import *
from tkinter import messagebox
from tkinter import ttk

# Database connection parameters
hostname = "localhost"
database = "postgres"
username = "postgres"
pwd = "gula"  # Change this to your actual password
port_id = 5432

def create_connection():
    try:
        conn = psycopg2.connect(
            host=hostname,
            dbname=database,
            user=username,
            password=pwd,
            port=port_id)
        return conn
    except Exception as e:
        messagebox.showerror("Database Error", f"Failed to connect to database: {str(e)}")
        return None

def setup_database():
    conn = create_connection()
    if conn:
        try:
            cur = conn.cursor()
            # Check if table exists
            cur.execute("SELECT EXISTS (SELECT FROM information_schema.tables WHERE table_name = 'students')")
            table_exists = cur.fetchone()[0]
            
            if table_exists:
                # Check if gender column exists
                cur.execute("""
                    SELECT EXISTS (
                        SELECT FROM information_schema.columns 
                        WHERE table_name='students' AND column_name='gender'
                    )
                """)
                column_exists = cur.fetchone()[0]
                
                if not column_exists:
                    # Add gender column if it doesn't exist
                    cur.execute("ALTER TABLE students ADD COLUMN gender VARCHAR")
                    conn.commit()
            else:
                # Create new table with gender column
                create_script = '''CREATE TABLE students (
                                    regnumber VARCHAR PRIMARY KEY, 
                                    firstname VARCHAR,
                                    lastname VARCHAR, 
                                    idnumber VARCHAR, 
                                    address VARCHAR, 
                                    emailaddress VARCHAR,
                                    level NUMERIC(3,1),
                                    programcode VARCHAR, 
                                    facultyname VARCHAR,
                                    gender VARCHAR)'''
                cur.execute(create_script)
                conn.commit()
            return conn
        except Exception as e:
            messagebox.showerror("Database Error", f"Failed to setup database: {str(e)}")
            return None

def submit_data():
    conn = create_connection()
    if conn:
        try:
            cur = conn.cursor()
            insert_script = '''INSERT INTO students(
                                regnumber, firstname, lastname, idnumber, 
                                address, emailaddress, level, programcode, facultyname, gender) 
                                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''
            
            # Validate required fields
            if not all([reg_entry.get(), firstname_entry.get(), lastname_entry.get(), id_entry.get()]):
                messagebox.showerror("Error", "Please fill all required fields")
                return
            
            values = (
                reg_entry.get(),
                firstname_entry.get(),
                lastname_entry.get(),
                id_entry.get(),
                physical_address_entry.get(),
                email_address_entry.get(),
                float(level_entry.get()) if level_entry.get() else None,
                program_name_entry.get(),
                faculty_entry.get(),
                gender_var.get()
            )
            
            cur.execute(insert_script, values)
            conn.commit()
            messagebox.showinfo("Success", "Student record added successfully!")
            clear()
        except ValueError:
            messagebox.showerror("Error", "Level must be a number (e.g. 2 or 2.2)")
        except Exception as e:
            conn.rollback()
            messagebox.showerror("Database Error", f"Failed to insert record: {str(e)}")
        finally:
            conn.close()

def show_data():
    conn = create_connection()
    if conn:
        try:
            cur = conn.cursor()
            cur.execute("SELECT * FROM students")
            records = cur.fetchall()
            
            # Clear previous data
            for item in tree.get_children():
                tree.delete(item)
                
            if records:
                for i, record in enumerate(records):
                    tag = 'evenrow' if i % 2 == 0 else 'oddrow'
                    tree.insert("", END, values=record, tags=(tag,))
            else:
                messagebox.showinfo("Info", "No records found")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to fetch data: {str(e)}")
        finally:
            conn.close()

def clear():
    reg_entry.delete(0, END)
    id_entry.delete(0, END)
    firstname_entry.delete(0, END)
    lastname_entry.delete(0, END)
    physical_address_entry.delete(0, END)
    email_address_entry.delete(0, END)
    level_entry.delete(0, END)
    program_name_entry.delete(0, END)
    faculty_entry.delete(0, END)
    gender_var.set("Male")  # Reset to default

# Main GUI Setup
root = Tk()
root.title("University of Zimbabwe Student System")
root.geometry("1350x800")
root.config(bg='khaki')

# Header
header = Label(root, text="UNIVERSITY OF ZIMBABWE", fg="#fff", bg='#f0687c', 
               font=("arial", 30, "bold"), bd=8, relief=GROOVE)
header.pack(fill=X, pady=10)

# Main Frame
main_frame = Frame(root, bg='khaki')
main_frame.pack(pady=10)

# Student Profile Frame
profile_frame = LabelFrame(main_frame, text="Student Profile", fg="black", bg='#f7f8de', 
                          font=("arial", 12, "bold"), bd=8, relief=GROOVE)
profile_frame.grid(row=0, column=0, padx=10, pady=10)

# Form Fields
fields = [
    ("Registration number:", 0, 0),
    ("ID number:", 0, 2),
    ("First name:", 1, 0),
    ("Last name:", 1, 2),
    ("Address:", 2, 0),
    ("Email:", 2, 2),
    ("Level (e.g. 2 or 2.2):", 3, 0),
    ("Program code:", 3, 2),
    ("Faculty:", 4, 0)
]

entries = []
for i, (text, row, col) in enumerate(fields):
    Label(profile_frame, text=text, bg="#f7f8de", font=("arial", 12)).grid(row=row, column=col, sticky="w", padx=5, pady=5)
    entry = Entry(profile_frame, width=25, font=("arial", 12), bd=3)
    entry.grid(row=row, column=col+1, padx=5, pady=5)
    entries.append(entry)

# Unpack entries
(reg_entry, id_entry, firstname_entry, lastname_entry, 
 physical_address_entry, email_address_entry, level_entry, 
 program_name_entry, faculty_entry) = entries

# Gender Selection
gender_var = StringVar(value="Male")
Label(profile_frame, text="Gender:", bg="#f7f8de", font=("arial", 12)).grid(row=4, column=2, sticky="w", padx=5, pady=5)
Radiobutton(profile_frame, text="Male", value="Male", variable=gender_var, bg="#f7f8de").grid(row=4, column=3, sticky="w")
Radiobutton(profile_frame, text="Female", value="Female", variable=gender_var, bg="#f7f8de").grid(row=4, column=4, sticky="w")

# Buttons Frame
button_frame = LabelFrame(main_frame, text="Actions", font=('arial', 14),
                        fg="purple", bg="#f7f8de")
button_frame.grid(row=1, column=0, pady=10)

Button(button_frame, text="Submit", bg="orange", font=("arial", 12), 
       width=12, command=submit_data).grid(row=0, column=0, padx=5, pady=5)
Button(button_frame, text="View Data", bg="orange", font=("arial", 12), 
       width=12, command=show_data).grid(row=0, column=1, padx=5, pady=5)
Button(button_frame, text="Clear", bg="orange", font=("arial", 12), 
       width=12, command=clear).grid(row=0, column=2, padx=5, pady=5)
Button(button_frame, text="Exit", bg="orange", font=("arial", 12), 
       width=12, command=root.destroy).grid(row=0, column=3, padx=5, pady=5)

# Output Frame with Treeview
output_frame = LabelFrame(root, text="Student Records", font=('arial', 14),
                        fg="purple", bg="#f7f8de")
output_frame.pack(fill=BOTH, expand=True, padx=10, pady=10)

# Treeview with scrollbars
tree_scroll_y = Scrollbar(output_frame)
tree_scroll_y.pack(side=RIGHT, fill=Y)

tree_scroll_x = Scrollbar(output_frame, orient=HORIZONTAL)
tree_scroll_x.pack(side=BOTTOM, fill=X)

tree = ttk.Treeview(output_frame, 
                   yscrollcommand=tree_scroll_y.set,
                   xscrollcommand=tree_scroll_x.set,
                   selectmode="extended")
tree.pack(fill=BOTH, expand=True)

tree_scroll_y.config(command=tree.yview)
tree_scroll_x.config(command=tree.xview)

# Define columns
tree["columns"] = ("Reg#", "First Name", "Last Name", "ID#", "Address", 
                   "Email", "Level", "Program", "Faculty", "Gender")

# Format columns
tree.column("#0", width=0, stretch=NO)
tree.column("Reg#", anchor=W, width=100)
tree.column("First Name", anchor=W, width=120)
tree.column("Last Name", anchor=W, width=120)
tree.column("ID#", anchor=W, width=120)
tree.column("Address", anchor=W, width=100)
tree.column("Email", anchor=W, width=180)
tree.column("Level", anchor=CENTER, width=80)
tree.column("Program", anchor=W, width=100)
tree.column("Faculty", anchor=W, width=120)
tree.column("Gender", anchor=CENTER, width=80)

# Create headings
tree.heading("#0", text="", anchor=W)
tree.heading("Reg#", text="Reg#", anchor=W)
tree.heading("First Name", text="First Name", anchor=W)
tree.heading("Last Name", text="Last Name", anchor=W)
tree.heading("ID#", text="ID#", anchor=W)
tree.heading("Address", text="Address", anchor=W)
tree.heading("Email", text="Email", anchor=W)
tree.heading("Level", text="Level", anchor=CENTER)
tree.heading("Program", text="Program", anchor=W)
tree.heading("Faculty", text="Faculty", anchor=W)
tree.heading("Gender", text="Gender", anchor=CENTER)

# Add striped row tags
tree.tag_configure('oddrow', background="white")
tree.tag_configure('evenrow', background="lightblue")

# Initialize database
setup_database()

root.mainloop()